In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Paths to dataset directories
folder_train = "/content/drive/MyDrive/project folder/data3a/training"
folder_test = "/content/drive/MyDrive/project folder/data3a/test"
image_size = 224  # VGG16 input size
batch_size = 32
epochs = 45  # Adjust as needed

In [ ]:
# ✅ Data Augmentation & Normalization
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1.0 / 255.0,  # Normalize pixel values
    validation_split=0.2  # Splitting training data into train/val
)

# ✅ Training Data Generator
train_generator = datagen.flow_from_directory(
    folder_train,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    folder_train,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# ✅ Test Data Generator (No augmentation, only normalization)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_directory(
    folder_test,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1108 images belonging to 3 classes.
Found 275 images belonging to 3 classes.
Found 248 images belonging to 3 classes.


In [ ]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# ✅ Fine-tuning: Unfreeze last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False  # Freeze all layers except last 4

In [ ]:
# ✅ Custom Classification Layers
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Increased Dropout

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Increased Dropout

output = Dense(3, activation='softmax')(x)  # 3 Classes: minor, moderate, severe

In [ ]:
# ✅ Create & Compile Model
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# ✅ Callbacks to Reduce Overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6)


In [ ]:
# ✅ Train the Model
model_info = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[reduce_lr]
)

Epoch 1/45
35/35 ━━━━━━━━━━━━━━━━━━━━ 33s 677ms/step - accuracy: 0.5031 - loss: 1.4156 - val_accuracy: 0.4582 - val_loss: 1.5556 - learning_rate: 1.0000e-04
Epoch 2/45
35/35 ━━━━━━━━━━━━━━━━━━━━ 22s 550ms/step - accuracy: 0.4996 - loss: 1.3879 - val_accuracy: 0.5273 - val_loss: 1.7354 - learning_rate: 1.0000e-04
Epoch 3/45
35/35 ━━━━━━━━━━━━━━━━━━━━ 23s 561ms/step - accuracy: 0.5480 - loss: 1.2591 - val_accuracy: 0.3673 - val_loss: 3.0117 - learning_rate: 1.0000e-04
Epoch 4/45
35/35 ━━━━━━━━━━━━━━━━━━━━ 41s 544ms/step - accuracy: 0.5445 - loss: 1.2828 - val_accuracy: 0.4545 - val_loss: 2.3676 - learning_rate: 1.0000e-04
Epoch 5/45
35/35 ━━━━━━━━━━━━━━━━━━━━ 43s 593ms/step - accuracy: 0.5043 - loss: 1.3291 - val_accuracy: 0.3564 - val_loss: 4.3542 - learning_rate: 1.0000e-04
Epoch 6/45
35/35 ━━━━━━━━━━━━━━━━━━━━ 39s 556ms/step - accuracy: 0.5569 - loss: 1.2250 - val_accuracy: 0.3818 - val_loss: 3.7936 - learning_rate: 1.0000e-04
Epoch 7/45
35/35 ━━━━━━━━━━━━━━━━━━━━ 23s 583ms/step - acc

In [ ]:
model.save("model.h5")

In [ ]:
# ✅ Evaluate on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 96s 14s/step - accuracy: 0.6504 - loss: 0.8828
Test Accuracy: 0.6613
